In [16]:
import pandas as pd
import datetime
import re
import time

In [17]:
def remove_str(row):
    x = row.Comments2
    try:
        x = x.replace(re.search('Originally posted .*', x)[0], '').strip()
    except:
        pass
    try:
        x = x.replace(re.search('Release Date:.*', x)[0], '').strip()
    except:
        pass
    row.Comments2 = x
    return row

Previous Year's Number of Post

In [18]:
def prev_year(date_temp):
    return df1[(((df1.Date - date_temp).astype('timedelta64[h]')/24) >= -365) & (((df1.Date - date_temp).astype('timedelta64[h]')/24) <=0)].shape[0]

90 days post number

In [19]:
def num_of_post(filtered_df):
    date_90_len = filtered_df.shape[0]
    return date_90_len

Remove Unwanted Lines from Comments

In [20]:
# filtered_df = filtered_df.apply(remove_str, axis='columns')

Row containing at least one negative words

In [21]:
def row_containing_neg_words(filtered_df, keyword):
    def func_inner(row):      
        for each_word in keyword:
            if each_word in str(row.Comments2):
                return 1
        return 0
    return sum(filtered_df.apply(func_inner, axis='columns'))

Number of Negative Words Mentioned

In [22]:
def criticism_words(filtered_df, keyword):
    def func2(row):
        word_count = 0
        for each in keyword:
            word_count += len(re.findall(each.lower(), str(row.Comments2)))
        return word_count
    return sum(filtered_df.apply(func2, axis='columns'))

Total Number of Words

In [23]:
def words_length(filtered_df):
    def func3(row):
        return len(str(row.Comments2).split())
    return sum(filtered_df.apply(func3, axis='columns'))

Function that manage program flow

In [24]:
def process_function(row): 
    keyword = ['am', 'is', 'are', 'was', 'were']
    row['Total Post in a Financial Year'] = prev_year(row['Year'])
    filtered_df = df1[(abs((df1.Date - row['Year']).astype('timedelta64[h]')/24) <= 90)]
    # print(filtered_df.isna().any())
    row['Total post related to earnings'] = num_of_post(filtered_df)
    row['Criticism Comments'] = row_containing_neg_words(filtered_df, keyword)
    row['Criticism Words'] = criticism_words(filtered_df, keyword)
    row['Total no. of words'] = words_length(filtered_df)
    return row


Main Loop

In [25]:
import os
try:
    os.chdir('New folder')
except:
    pass
files = [i for i in os.listdir() if '.xlsx' in i]
for i in range(0, len(files), 2):
    # Read Excel File
    df1 = pd.read_excel(files[i])
    df2 = pd.read_excel(files[i+1])
    # Fill Empty Cell
    df1.Date = df1.Date.fillna(method='ffill')
    df1 = df1.fillna('')
    #Convert to Datetime
    df1.Date = pd.to_datetime(df1.Date)
    df2.Year = pd.to_datetime(df2.Year, format=r'%Y-%m-%d')
    df2 = df2.apply(process_function, axis='columns')
    df2.to_csv(str(i) + '.csv', index=False)